In [129]:
from qiskit import *
import numpy as np
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
from qiskit_aer import AerSimulator
from qiskit.circuit.library.standard_gates import XGate, RYGate
import qiskit.quantum_info as qi
from qutip import *
np.set_printoptions(precision = 1)

I defined my functions that calculate the Hilbert-Space and I used Qutip to handle some simulatory work

In [152]:
def HilSch(matrix_1, matrix_2, n):
    Obj1 = Qobj(matrix_1)
    Obj2 = Qobj(matrix_2)
    return np.sqrt(1-(abs((Obj1.dag()*Obj2).tr())**2)/(2**(2*n)))

def MatrixConvertor(circuit):
    return np.matrix(qi.Operator(circuit))

Defined the Toffoli matrix to compare to

In [153]:
Toff = QuantumCircuit(3)
Toff.ccx(0,1,2)
Toff.draw()

q_0: ──■──
       │  
q_1: ──■──
     ┌─┴─┐
q_2: ┤ X ├
     └───┘

In [154]:
Toffoli = MatrixConvertor(Toff)
print (Toffoli)

[[1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 1.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]]


This circuit is a relative phase circuit and with just 3 two-qubit gates can produce a fairly close Toffoli. For many tries, this remained the bound with just 3-4 two-qubit gates

In [155]:
circ = QuantumCircuit(3)
circ.ry(np.pi/4, 2)
circ.cx(1,2)
circ.ry(np.pi/4, 2)
circ.cx(0,2)
circ.ry(-np.pi/4, 2)
circ.cx(1,2)
circ.ry(-np.pi/4, 2)
circ.draw(fold = -1, style = 'ipl')

q_0: ─────────────────────────────■───────────────────────────────
                                  │                               
q_1: ─────────────■───────────────┼────────────────■──────────────
     ┌─────────┐┌─┴─┐┌─────────┐┌─┴─┐┌──────────┐┌─┴─┐┌──────────┐
q_2: ┤ Ry(π/4) ├┤ X ├┤ Ry(π/4) ├┤ X ├┤ Ry(-π/4) ├┤ X ├┤ Ry(-π/4) ├
     └─────────┘└───┘└─────────┘└───┘└──────────┘└───┘└──────────┘

In [156]:
print (HilSch(MatrixConvertor(circ), Toffoli, 3))

0.6614378277661477


This is the circuit that Prof Jens' computed in class and evidently is closes to Toffoli with just four two-qubit states

In [163]:
circ5 = QuantumCircuit(3)
circ5.h(2)
circ5.cs(1,2)
circ5.cx(0,1)
circ5.csdg(1,2)
circ5.cx(0,1)
circ5.t([0,2])
circ5.h(2)
circ5.draw(fold = -1)

┌───┐
q_0: ────────────■───────────■──┤ T ├
               ┌─┴─┐       ┌─┴─┐└───┘
q_1: ───────■──┤ X ├───■───┤ X ├─────
     ┌───┐┌─┴─┐└───┘┌──┴──┐├───┤┌───┐
q_2: ┤ H ├┤ S ├─────┤ Sdg ├┤ T ├┤ H ├
     └───┘└───┘     └─────┘└───┘└───┘

In [164]:
print (HilSch(MatrixConvertor(circ5), Toffoli, 3))

0.38268343236509084


This circuit is from Neilson and Chuang and does use extra two-qubit gates, I used it to check if my definitions were correct and to see how close people have really got with free liberty of choosing how many every two-qubit states they need.

In [165]:
circ3 = QuantumCircuit(3)
circ3.h(2)
circ3.cx(1,2)
circ3.tdg(2)
circ3.cx(0,2)
circ3.t(2)
circ3.cx(1,2)
circ3.tdg(2)
circ3.cx(0,2)
circ3.t([1,2])
circ3.cx(0, 1)
circ3.tdg(1)
circ3.t(0)
circ3.cx(0,1)
circ3.h(2)
circ3.draw(fold = -1)

┌───┐      
q_0: ───────────────────■─────────────────────■────■───┤ T ├───■──
                        │             ┌───┐   │  ┌─┴─┐┌┴───┴┐┌─┴─┐
q_1: ───────■───────────┼─────────■───┤ T ├───┼──┤ X ├┤ Tdg ├┤ X ├
     ┌───┐┌─┴─┐┌─────┐┌─┴─┐┌───┐┌─┴─┐┌┴───┴┐┌─┴─┐├───┤└┬───┬┘└───┘
q_2: ┤ H ├┤ X ├┤ Tdg ├┤ X ├┤ T ├┤ X ├┤ Tdg ├┤ X ├┤ T ├─┤ H ├──────
     └───┘└───┘└─────┘└───┘└───┘└───┘└─────┘└───┘└───┘ └───┘

In [166]:
print (HilSch(MatrixConvertor(circ3), Toffoli, 3))

2.1073424255447017e-08


From here, I reduced their circuit by two two-qubit CX gates and got a circuit that is better than relative phase circuits but still worse than Prof Jens' CS gate circuit. With a Hilbert-Schmidt distance of 0.52

In [180]:
circ3 = QuantumCircuit(3)
circ3.h(2)
circ3.cx(1,2)
circ3.tdg(2)
circ3.cx(0,2)
circ3.t(2)
circ3.cx(1,2)
circ3.tdg(2)
circ3.cx(0,2)
circ3.t(2)
circ3.t(0)
circ3.h(2)
circ3.draw(fold = -1)

┌───┐     
q_0: ───────────────────■─────────────────────■──┤ T ├─────
                        │                     │  └───┘     
q_1: ───────■───────────┼─────────■───────────┼────────────
     ┌───┐┌─┴─┐┌─────┐┌─┴─┐┌───┐┌─┴─┐┌─────┐┌─┴─┐┌───┐┌───┐
q_2: ┤ H ├┤ X ├┤ Tdg ├┤ X ├┤ T ├┤ X ├┤ Tdg ├┤ X ├┤ T ├┤ H ├
     └───┘└───┘└─────┘└───┘└───┘└───┘└─────┘└───┘└───┘└───┘

In [181]:
print (HilSch(MatrixConvertor(circ3), Toffoli, 3))

0.5210053832799876
